In [ ]:
import SimpleITK as sitk
import os
import os
import six
import csv
from PIL import Image
import radiomics
from radiomics import featureextractor  # This module is used for interaction with pyradiomics
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import f_classif
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import warnings

In [ ]:
#读取excel内部数据，将癌症种类和总文件夹中的1-77数字标签对应
Label_preprocess_data = pd.read_excel("/root/Split_data_"+center_name +".xlsx", usecols=[6, 10], nrows=77)  #含有每个数据标签的数据
Label_preprocess_data_todict = Label_preprocess_data.to_dict('list')
pizhiai_patient_folder = []
shigeliu_patient_folder = []
xianliu_patient_folder = []
total_patient_folder = []
count = 0
for i in Label_preprocess_data_todict['病理结果']:
    total_patient_folder.append(Label_preprocess_data_todict['编号'][count])
    if i == '皮质癌':
      pizhiai_patient_folder.append(Label_preprocess_data_todict['编号'][count])
    if i == '嗜铬细胞瘤':
      shigeliu_patient_folder.append(Label_preprocess_data_todict['编号'][count])
    count = count + 1
 

In [3]:
def resample_img(itk_image, out_spacing=[1.0, 1.0, 1.0], is_label=False):
    
    # Resample images to 1mm spacing with SimpleITK
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkBSpline) #三次样条插

    return resample.Execute(itk_image)

In [4]:
Total_file_path = "/root/data_"+center_name+"/Split_data_"+center_name  # 总的数据集的路径

In [ ]:
#保证数据有正确的scaler
wrong_data_location_image = []
wrong_data_location_mask = []
total_patient_folder_new = [str(x) for x in total_patient_folder]
for patient in total_patient_folder_new :
  patient_path = Total_file_path + '/' + patient
  fenqi_list_pre = [i for i in os.listdir(patient_path) if 'nrrd' not in i]
  fenqi_list= [j for j in fenqi_list_pre if not j.startswith('.')]
  fenqi_list.sort()
  for fenqi in fenqi_list:
      data_path = patient_path + '/' + fenqi
      print(data_path)
      data_list_pre = os.listdir(data_path)
      data_list = [j for j in data_list_pre if not j.startswith('.')]
      data_list.sort()
      image = sitk.ReadImage(data_path + '/' + data_list[1], imageIO="NrrdImageIO")
      mask = sitk.ReadImage(data_path + '/' + data_list[0], imageIO="NrrdImageIO")
      #print(data_path + '/' + data_list[1])
      if image.GetNumberOfComponentsPerPixel() == 3:
        wrong_data_location_image.append(data_path + '/' + data_list[1])
      if mask.GetNumberOfComponentsPerPixel() == 3:
        wrong_data_location_mask.append(data_path + '/' + data_list[0])

In [6]:
pizhiai_path =  "/root/pizhiai"
shigexibaoliu_path = "/root/shigexibaoliu"
#对皮质癌的路径
patient_list_pre_pz = [i for i in os.listdir(pizhiai_path) if 'xlsx' not in i]#返回指定路径下的文件和文件夹列表
patient_list_pz = [f for f in patient_list_pre_pz if not f.startswith('.')]#删去苹果自带的'.DS_Store'文件
patient_list_pz.sort(key=lambda x:int(x))
#对shige细胞瘤的路径
patient_list_pre_sg = [i for i in os.listdir(shigexibaoliu_path) if 'xlsx' not in i]#返回指定路径下的文件和文件夹列表
patient_list_sg = [f for f in patient_list_pre_sg if not f.startswith('.')]#删去苹果自带的'.DS_Store'文件
patient_list_sg.sort(key=lambda x:int(x))

In [ ]:
#保证数据有正确的scaler 皮质癌
wrong_data_location_image = []
wrong_data_location_mask = []
for patient in patient_list_pz :
  patient_path = pizhiai_path + '/' + patient
  fenqi_list_pre = [i for i in os.listdir(patient_path) if 'nrrd' not in i]
  fenqi_list= [j for j in fenqi_list_pre if not j.startswith('.')]
  fenqi_list.sort()
  for fenqi in fenqi_list:
      data_path = patient_path + '/' + fenqi
      print(data_path)
      data_list_pre = os.listdir(data_path)
      data_list = [j for j in data_list_pre if not j.startswith('.')]
      data_list.sort()
      image = sitk.ReadImage(data_path + '/' + data_list[1], imageIO="NrrdImageIO")
      mask = sitk.ReadImage(data_path + '/' + data_list[0], imageIO="NrrdImageIO")
      #print(data_path + '/' + data_list[1])
      if image.GetNumberOfComponentsPerPixel() == 3:
        wrong_data_location_image.append(data_path + '/' + data_list[1])
      if mask.GetNumberOfComponentsPerPixel() == 3:
        wrong_data_location_mask.append(data_path + '/' + data_list[0])

In [ ]:
#保证数据有正确的scaler #嗜铬细胞瘤
wrong_data_location_image = []
wrong_data_location_mask = []
for patient in patient_list_sg :
  patient_path = shigexibaoliu_path + '/' + patient
  fenqi_list_pre = [i for i in os.listdir(patient_path) if 'nrrd' not in i]
  fenqi_list= [j for j in fenqi_list_pre if not j.startswith('.')]
  fenqi_list.sort()
  for fenqi in fenqi_list:
      data_path = patient_path + '/' + fenqi
      print(data_path)
      data_list_pre = os.listdir(data_path)
      data_list = [j for j in data_list_pre if not j.startswith('.')]
      data_list.sort()
      image = sitk.ReadImage(data_path + '/' + data_list[1], imageIO="NrrdImageIO")
      mask = sitk.ReadImage(data_path + '/' + data_list[0], imageIO="NrrdImageIO")
      #print(data_path + '/' + data_list[1])
      if image.GetNumberOfComponentsPerPixel() == 3:
        wrong_data_location_image.append(data_path + '/' + data_list[1])
      if mask.GetNumberOfComponentsPerPixel() == 3:
        wrong_data_location_mask.append(data_path + '/' + data_list[0])

In [ ]:
Feature_Table_pz = {}
patient_list_pre_pz = [i for i in os.listdir(pizhiai_path) if 'xlsx' not in i]#返回指定路径下的文件和文件夹列表
patient_list_pz = [f for f in patient_list_pre_pz if not f.startswith('.')]#删去苹果自带的'.DS_Store'文件
patient_list_pz.sort(key=lambda x:int(x))
for patient_pz in patient_list_pz:
  patient_path_pz = pizhiai_path + '/' + patient_pz
  fenqi_list_pre_pz = [i for i in os.listdir(patient_path_pz) if 'nrrd' not in i]
  fenqi_list_pz = [j for j in fenqi_list_pre_pz if not j.startswith('.')]
  fenqi_list_pz.sort()
  for fenqi_pz in fenqi_list_pz:
      data_path_pz = patient_path_pz + '/' + fenqi_pz
      data_list_pre_pz = os.listdir(data_path_pz)
      data_list_pz = [j for j in data_list_pre_pz if not j.startswith('.')]
      data_list_pz.sort()
      image_pz = sitk.ReadImage(data_path_pz + '/' + data_list_pz[1], imageIO="NrrdImageIO")
      mask_pz = sitk.ReadImage(data_path_pz + '/' + data_list_pz[0], imageIO="NrrdImageIO")
      # Resample重采样
      respacing_image_pz = resample_img(image_pz, [1.0, 1.0, 1.0], is_label=False)
      respacing_mask_pz = resample_img(mask_pz, [1.0, 1.0, 1.0], is_label=True)
      rif = sitk.ResampleImageFilter()
      rif.SetReferenceImage(respacing_image_pz)
      rif.SetOutputPixelType(respacing_mask_pz.GetPixelID())
      rif.SetInterpolator(sitk.sitkNearestNeighbor)
      respacing_remask_pz = rif.Execute(respacing_mask_pz) #最后能用的是respacing_image_pz,和respacing_remask_pz

      extractor_pz = featureextractor.RadiomicsFeatureExtractor(respacing_image_pz, respacing_remask_pz)
      extractor_pz.enableImageTypeByName('Wavelet')
      extractor_pz.enableImageTypeByName('LoG', customArgs={'sigma':[0.5, 1.0, 1.5, 2.0, 2.5, 3.0]}) 
      ordered_result_pz = extractor_pz.execute(respacing_image_pz, respacing_remask_pz) 
      result_pz = dict(ordered_result_pz)
      Feature_Table_pz['patient_id'] = patient_pz 
      Feature_Table_pz['phase'] = '\'' + fenqi_pz #分期号
      Feature_Table_pz.update(result_pz) #把result字典添加到指定的feature_table字典里
      #数据写入exc
      with open("/root/pizhiai_new.csv", 'a', newline = '') as csvfile:
          writer = csv.DictWriter(csvfile, fieldnames=Feature_Table_pz.keys())
          file_is_empty = os.stat("/root/pizhiai_new.csv").st_size == 0
          if file_is_empty: # 解决表头在每个循环构建的字典内多次复写的方法
            writer.writeheader()
          writer.writerow(Feature_Table_pz)

In [ ]:
Feature_Table_sg = {}
patient_list_pre_sg = [i for i in os.listdir(shigexibaoliu_path) if 'xlsx' not in i]#返回指定路径下的文件和文件夹列表
patient_list_sg = [f for f in patient_list_pre_sg if not f.startswith('.')]#删去苹果自带的'.DS_Store'文件
patient_list_sg.sort(key=lambda x:int(x))
for patient_sg in patient_list_sg:
  patient_path_sg = shigexibaoliu_path + '/' + patient_sg
  fenqi_list_pre_sg = [i for i in os.listdir(patient_path_sg) if 'nrrd' not in i]
  fenqi_list_sg = [j for j in fenqi_list_pre_sg if not j.startswith('.')]
  fenqi_list_sg.sort()
  for fenqi_sg in fenqi_list_sg:
      data_path_sg = patient_path_sg + '/' + fenqi_sg
      data_list_pre_sg = os.listdir(data_path_sg)
      data_list_sg = [j for j in data_list_pre_sg if not j.startswith('.')]
      data_list_sg.sort()
      image_sg = sitk.ReadImage(data_path_sg + '/' + data_list_sg[1], imageIO="NrrdImageIO")
      mask_sg = sitk.ReadImage(data_path_sg + '/' + data_list_sg[0], imageIO="NrrdImageIO")
      # Resample重采样
      respacing_image_sg = resample_img(image_sg, [1.0, 1.0, 1.0], is_label=False)
      respacing_mask_sg = resample_img(mask_sg, [1.0, 1.0, 1.0], is_label=True)
      rif = sitk.ResampleImageFilter()
      rif.SetReferenceImage(respacing_image_sg)
      rif.SetOutputPixelType(respacing_mask_sg.GetPixelID())
      rif.SetInterpolator(sitk.sitkNearestNeighbor)
      respacing_remask_sg = rif.Execute(respacing_mask_sg) #最后能用的是respacing_image_pz,和respacing_remask_pz

      extractor_sg = featureextractor.RadiomicsFeatureExtractor(respacing_image_sg, respacing_remask_sg)
      extractor_sg.enableImageTypeByName('Wavelet')
      extractor_sg.enableImageTypeByName('LoG', customArgs={'sigma':[0.5, 1.0, 1.5, 2.0, 2.5, 3.0]}) 
      ordered_result_sg = extractor_sg.execute(respacing_image_sg, respacing_remask_sg) 
      result_sg = dict(ordered_result_sg)
      Feature_Table_sg['patient_id'] = patient_sg 
      Feature_Table_sg['phase'] = '\'' + fenqi_sg #分期号
      Feature_Table_sg.update(result_sg) #把result字典添加到指定的feature_table字典里
      #数据写入exc
      with open("/root/shigeliu_new.csv", 'a', newline = '') as csvfile:
          writer = csv.DictWriter(csvfile, fieldnames=Feature_Table_sg.keys())
          file_is_empty = os.stat("/root/shigeliu_new.csv").st_size == 0
          if file_is_empty: # 解决表头在每个循环构建的字典内多次复写的方法
            writer.writeheader()
          writer.writerow(Feature_Table_sg)